In [38]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [40]:
download = drive.CreateFile({'id': '1Hydbg7KXOoK9Xf-DPFsoMGv6Whdzhh1F'})
download.GetContentFile('train_full.csv')
download = drive.CreateFile({'id': '1uo-5rbDJKUSBondu2NnaMC4ygu1TwuW0'})
download.GetContentFile('test_full.csv')

2019-04-19 09:41:19,934 : WARNING : file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No mo

In [0]:
import pandas as pd
import numpy as np

label_name = 'non-toxic'

train_full = pd.read_csv('train_full.csv')
test_full = pd.read_csv('test_full.csv')



In [6]:
more_rows = test_full.iloc[0:4000,:]
test_full = test_full.drop(test_full.index[0:4000])

train_full = pd.concat([train_full,more_rows],axis='rows',ignore_index=True).reset_index(drop=True)
print(train_full.shape,test_full.shape)

(23993, 9) (6998, 9)


Create word vectors

In [0]:
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report

In [0]:
train = train_full["comment_text"]
test = test_full["comment_text"]

trainlab = train_full[label_name]
testlab = test_full[label_name]

In [44]:
# string cleaning- already done
# Download and load the punkt tokenizer for sentence splitting
import nltk.data
nltk.download("popular")   
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [0]:
# to split a review into parsed sentences returning a list of sentences (each sentence is list of words)
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    review = str(review)
    raw_sentences = tokenizer.tokenize(review.strip()) # split para into sentences
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            #sentences.append( review_to_wordlist(raw_sentence,remove_stopwords) )
            sentences.append(raw_sentence) #bcz already cleaned h
    return sentences

In [46]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for r in train:
  sentences += review_to_sentences(r, tokenizer)

print ("Parsing sentences from testing set")
for r in test:
    sentences += review_to_sentences(r, tokenizer)
    
print(sentences[0:5])

Parsing sentences from training set
Parsing sentences from testing set
['guys fuckin crazy stupid shit lick pimply fat ugly bum crazy gays faggots male lesbian know gonna delete stupid block thing care single weirdest boobs okay hate bitches', 'guys victimizing ethnicity blatant racism', 'delete harry walther article worried deletion walther one voices preaching truth jesus message', 'think give shit', 'disagree fact used single party government largely irrelvance important thing still constitutional monarchy parliamentary system allows coalitions majority rule changed say currently coalition whatever party means position deputy prime minister included alongside heads state government false mentioning nick clegg deputy prime minister article around place david cameron currently described prime minister put clegg alongside queen prime minister basis coalition government right job constitutionally defined prime minister second command cameron die resign suddenly clegg would become prime 

In [0]:
#training and saving model 
import logging # for nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [48]:
# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
div_factor = 1000    # division factor for clusters
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

2019-04-19 09:41:26,238 : INFO : collecting all words and their counts
2019-04-19 09:41:26,240 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2019-04-19 09:41:26,242 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2019-04-19 09:41:26,517 : INFO : PROGRESS: at sentence #10000, processed 2216528 words, keeping 27 word types
2019-04-19 09:41:26,796 : INFO : PROGRESS: at sentence #20000, processed 4506219 words, keeping 27 word types
2019-04-19 09:41:27,072 : INFO : PROGRESS: at sentence #30000, processed 6732145 words, keeping 27 word types
2019-04-19 09:41:27,100 : INFO : collected 27 word types from a corpus of 6929356 raw words and 30991 sentences
2019-04-19 09:41:27,101 : INFO : Loading a fresh vocabulary
2019-04-19 09:41:27,102 : INFO : effective_min_count=40 retains 27 unique words (100% of original 27, drops 0)
2019-04-19 09:41:27,103 : INFO : effective_min_count=40 leaves 6929356 word corpus (100% of original 6929356, drops 0)
2019-04-19 09:41:27,104 : INFO : deleting the raw counts dictionary of 27 items
2019-04-19 09:41:27,105 : INFO : sample=0.001 downsamples 23 most-common words
2019-04-19 09:41:27,107 : INFO : downsampling leaves estimated 1194589 word corpus (17.2% of prior 6929356)
2

Converting review to word vectors

In [0]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, num_features)
       print(reviewFeatureVecs)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [50]:
trainDataVecs = getAvgFeatureVecs( train, model, num_features )
testDataVecs = getAvgFeatureVecs( test, model, num_features )


Output hidden; open in https://colab.research.google.com to view.

Clustering these vectors

In [51]:
type(trainDataVecs)
print(trainDataVecs.shape)
tot = len(trainDataVecs)+len(testDataVecs)
print(tot)

arr = np.zeros((tot, num_features))

for col in range(0,num_features):
  for row in range(0, len(trainDataVecs)):
    arr[row] = np.array(trainDataVecs[row])
  
  
print(arr.shape)
l1 = len(trainDataVecs)
l2 = len(testDataVecs)
  
  
for col in range(0,num_features):
  for row in range(0, len(testDataVecs)):
    arr[l1 + row] = np.array(testDataVecs[row])


#word_vectors = np.concatenate(trainDataVecs,testDataVecs)
#print(word_vectors.shape)


(19993, 100)
30991
(30991, 100)


In [52]:
word_vectors = np.matrix(arr)
print(word_vectors.shape)

(30991, 100)


In [53]:
#word_vectors = pd.concat([trainDataVecs,testDataVecs])
#num_clusters = int(len(word_vectors) / div_factor)
#from sklearn.cluster import DBSCAN


num_clusters = 50

# Initalize a k-means object and use it to extract centroids
import time

start = time.time() # Start time


clusterer = KMeans( n_clusters = num_clusters, max_iter=500,  n_init=20)

idx = clusterer.fit_predict( word_vectors )


# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: ", elapsed, "seconds.")
print (idx)

Time taken for K Means clustering:  97.80170440673828 seconds.
[10 26 34 ... 18 17 43]


BAG OF CENTROIDS


In [0]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [55]:
# For the first 10 clusters
for cluster in range(0,50):
    #
    # Print the cluster number  
    print ("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print (words)


Cluster 0
[]

Cluster 1
['s', 'l', 'u', 'q']

Cluster 2
['a']

Cluster 3
['p', 'h', 'z']

Cluster 4
[]

Cluster 5
['m']

Cluster 6
[]

Cluster 7
[]

Cluster 8
[]

Cluster 9
[]

Cluster 10
[' ', 'c']

Cluster 11
[]

Cluster 12
[]

Cluster 13
[]

Cluster 14
[]

Cluster 15
[]

Cluster 16
['n']

Cluster 17
[]

Cluster 18
[]

Cluster 19
['r']

Cluster 20
['y']

Cluster 21
[]

Cluster 22
[]

Cluster 23
[]

Cluster 24
['x']

Cluster 25
[]

Cluster 26
['e', 'o', 'd', 'j']

Cluster 27
[]

Cluster 28
[]

Cluster 29
['b']

Cluster 30
[]

Cluster 31
[]

Cluster 32
[]

Cluster 33
[]

Cluster 34
['i', 'f']

Cluster 35
[]

Cluster 36
[]

Cluster 37
[]

Cluster 38
[]

Cluster 39
['v']

Cluster 40
[]

Cluster 41
['t', 'g', 'k']

Cluster 42
[]

Cluster 43
[]

Cluster 44
[]

Cluster 45
[]

Cluster 46
['w']

Cluster 47
[]

Cluster 48
[]

Cluster 49
[]


In [0]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = num_clusters
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    #print(len(bag_of_centroids))
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    #print(len(bag_of_centroids))
    return bag_of_centroids

In [0]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train.size, num_clusters), dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in train:
    train_centroids[counter][:] =  create_bag_of_centroids( review, word_centroid_map ) 
    counter = counter + 1

# Repeat for test reviews 
test_centroids = np.zeros(( test.size, num_clusters), dtype="float32" )

counter = 0
for review in test:
    test_centroids[counter] = create_bag_of_centroids( review, word_centroid_map )
    counter += 1

Random Forest Classifier

In [58]:
from sklearn.ensemble import RandomForestClassifier
# Fit a random forest and extract predictions 
forest = RandomForestClassifier()

# Fitting the forest may take a few minutes
print ("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train_full["non-toxic"])
result = forest.predict(test_centroids)

Fitting a random forest to labeled training data...


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [59]:
from sklearn.metrics import classification_report
#test data
print(classification_report(testlab, result))

              precision    recall  f1-score   support

           0       0.57      0.71      0.63      5000
           1       0.69      0.55      0.62      5998

   micro avg       0.62      0.62      0.62     10998
   macro avg       0.63      0.63      0.62     10998
weighted avg       0.64      0.62      0.62     10998



SVM poly

In [0]:
from sklearn.svm import SVC
# Fit a random forest and extract predictions 
forest = SVC(kernel = 'poly')

# Fitting the forest may take a few minutes
print ("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train_full["non-toxic"])
result = forest.predict(test_centroids)

# Write the test results 

Fitting a random forest to labeled training data...


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [0]:
from sklearn.metrics import classification_report
#test data
print(classification_report(testlab, result))